<a href="https://colab.research.google.com/github/laskari/END-Program/blob/main/Week-9/Model_1_SQuaD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

train_data = pd.read_json('/content/drive/MyDrive/END School of AI/Datasets/END Datasets/SQuad/train-v2.0.json', lines=False)
dev_data = pd.read_json('/content/drive/MyDrive/END School of AI/Datasets/END Datasets/SQuad/dev-v2.0.json', lines = False)

In [2]:
train_data.head()

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [3]:
def data_preperation(path):
  context = []
  answer_start =[]
  question = []
  answer = []

  for i in range(path.shape[0]):
    paragh = path.data[i]['paragraphs']

    for p in paragh:
      qas = p['qas']
      for qa in qas:
        if qa['answers']:

          question.append(qa['question'])
          answer_start.append(qa['answers'][0]['answer_start'])
          answer.append(qa['answers'][0]['text'])
          context.append(p['context'])


  squad_df = pd.DataFrame({'Context':context, 'question':question, 'answer_start': answer_start, 'answer':answer})
  return squad_df

In [4]:
squad_train = data_preperation(train_data)
squad_dev = data_preperation(dev_data)

In [5]:
squad_train.head(2)

,Context,question,answer_start,answer
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,269,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,207,singing and dancing


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.data import Field, BucketIterator
from torchtext import data
import spacy

import numpy as np
import random
import math
import time

In [7]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

torch.backends.cudnn.deterministic = True

In [8]:
SRC = Field(tokenize='spacy',
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize='spacy',
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

In [9]:
fields = [('question', SRC),('answer',TRG)]
example = [data.Example.fromlist([squad_train.question[i],squad_train.answer[i]], fields) for i in range(squad_train.shape[0])] 
QADataset = data.Dataset(example, fields)


dev_example = example = [data.Example.fromlist([squad_dev.question[i],squad_dev.answer[i]], fields) for i in range(squad_dev.shape[0])] 
valid = data.Dataset(dev_example, fields)

In [10]:
(train, test) = QADataset.split(split_ratio=[0.90, 0.10], random_state=random.seed(SEED))
print(len(train), len(valid), len(test))

78139 5928 8682


In [11]:
SRC.build_vocab(train, min_freq = 2)
TRG.build_vocab(train, min_freq = 2)

In [12]:
print("size of SRC vocab: ", len(SRC.vocab))
print('size of TRG vocab: ', len(TRG.vocab))

size of SRC vocab:  20179
size of TRG vocab:  17884


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
train_iterator, valid_iterator = BucketIterator.splits((train, valid),sort = False, batch_size = BATCH_SIZE, device = device)

In [14]:
# Encoder
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, dropout):
    super().__init__()
    self.hid_dim = hid_dim
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, hid_dim)
    self.dropout = nn.Dropout(dropout)
  def forward(self, src):
    embedded = self.dropout(self.embedding(src))

    outputs, hidden = self.rnn(embedded)
      
    #outputs = [src len, batch size, hid dim * n directions]
    #hidden = [n layers * n directions, batch size, hid dim]
    return hidden

In [15]:
# Decoder 
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, dropout):
    super().__init__()
    self.hid_dim = hid_dim,
    self.output_dim = output_dim
    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
    self.fc = nn.Linear(emb_dim + hid_dim * 2, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, context):
    #input = [batch size]
    #hidden = [n layers * n directions, batch size, hid dim]
    #context = [n layers * n directions, batch size, hid dim]
        
    #n layers and n directions in the decoder will both always be 1, therefore:
    #hidden = [1, batch size, hid dim]
    #context = [1, batch size, hid dim]
    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))

    emb_con = torch.cat((embedded, context), dim = 2)
    output, hidden = self.rnn(emb_con, hidden)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
    output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)
    prediction = self.fc(output)
    
    return prediction, hidden

In [16]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    #assert encoder.hid_dim == decoder.hid_dim, \
    #  "Hidden dimensions of encoder and decoder must be equal"
  def forward(self, src, trg, teacher_forcing_ratio = 0.5):

    #src = [src len, batch size]
    #trg = [trg len, batch size]
    #teacher_forcing_ratio is probability to use teacher forcing
    #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

    batch_size = src.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
    context = self.encoder(src)
    hidden = context
    input = trg[0, :]
    for t in range(1, trg_len):
      output, hidden = self.decoder(input, hidden, context)
      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      input = trg[t] if teacher_force else top1
    
    return outputs

In [17]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [18]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(20179, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(17884, 256)
    (rnn): GRU(768, 512)
    (fc): Linear(in_features=1280, out_features=17884, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 35,805,404 trainable parameters


In [20]:
optimizer = optim.Adam(model.parameters())

In [21]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [22]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 3m 31s
	Train Loss: 6.334 | Train PPL: 563.602
	 Val. Loss: 5.753 |  Val. PPL: 315.147
Epoch: 02 | Time: 3m 34s
	Train Loss: 5.903 | Train PPL: 366.029
	 Val. Loss: 5.703 |  Val. PPL: 299.816
Epoch: 03 | Time: 3m 34s
	Train Loss: 5.745 | Train PPL: 312.479
	 Val. Loss: 5.732 |  Val. PPL: 308.601
Epoch: 04 | Time: 3m 34s
	Train Loss: 5.502 | Train PPL: 245.190
	 Val. Loss: 5.478 |  Val. PPL: 239.368
Epoch: 05 | Time: 3m 35s
	Train Loss: 5.168 | Train PPL: 175.526
	 Val. Loss: 5.352 |  Val. PPL: 210.928
Epoch: 06 | Time: 3m 35s
	Train Loss: 4.867 | Train PPL: 129.926
	 Val. Loss: 5.346 |  Val. PPL: 209.827
Epoch: 07 | Time: 3m 35s
	Train Loss: 4.521 | Train PPL:  91.906
	 Val. Loss: 5.385 |  Val. PPL: 218.021
